In [1]:
import numpy as np
from scipy.io import loadmat

import tensorflow as tf

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

response = loadmat("data/regression/response.mat")['response']
vel = loadmat("data/regression/velocity.mat")['velocity']

In [2]:
response_ = response[:, 1:, 0]
vel_ = vel[:, :, 0]

for i in range(1, 8):
    response_ = np.vstack([response_, response[:, 1:, i]])
    vel_ = np.vstack([vel_, vel[:, :, i]])
    
response_.shape

(15960, 98)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(response_, vel_, test_size=0.2, random_state=42, shuffle=True)
X_train, X_valid, y_train, y_valid = train_test_split(response_, vel_, test_size=0.1, random_state=42, shuffle=True)

print(f"X_train: {X_train.shape} X_valid: {X_valid.shape} X_test: {X_test.shape}")

X_train: (14364, 98) X_valid: (1596, 98) X_test: (3192, 98)


In [4]:
def print_metrics(y_test, pred):
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    
    print(f"MAE: {mae} MSE: {mse} R2: {r2}")

In [5]:
lm = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)
pred = lm.predict(X_test)

print_metrics(y_test, pred)

MAE: 0.1902411520479643 MSE: 0.06758910457899983 R2: 0.38975893562824326


In [6]:
svr = MultiOutputRegressor(SVR()).fit(X_train, y_train)
pred = svr.predict(X_test)

print_metrics(y_test, pred)

MAE: 0.1388870263507495 MSE: 0.03755577691284195 R2: 0.6585613031416631


In [7]:
tree = MultiOutputRegressor(DecisionTreeRegressor()).fit(X_train, y_train)
pred = tree.predict(X_test)

print_metrics(y_test, pred)

MAE: 0.12044890834879171 MSE: 0.061570229866166254 R2: 0.43926976902324416


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[98]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(2)
])

optimizer = tf.keras.optimizers.Adam(0.0001)

model.compile(loss='mae',
              optimizer=optimizer,
              metrics=['mae', 'mse'])

model.fit(X_train, y_train, epochs=50, verbose=0)
pred = model.predict(X_test)

print_metrics(y_test, pred)

2021-12-18 18:37:26.139576: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-18 18:37:26.139736: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


MAE: 0.19620818219749278 MSE: 0.09490343526935349 R2: 0.17659750412745112
